In [2]:
from __future__ import print_function, division

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np

class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid, target_label = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([noise, label], [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, 100)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.summary()

        img = Input(shape=self.img_shape)

        # Extract feature representation
        features = model(img)

        # Determine validity and label of the image
        validity = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes, activation="softmax")(features)

        return Model(img, [validity, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Configure inputs
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, 100))

            # The labels of the digits that the generator tries to create an
            # image representation of
            sampled_labels = np.random.randint(0, 10, (batch_size, 1))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, sampled_labels])

            # Image labels. 0-9 
            img_labels = y_train[idx]

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, [valid, img_labels])
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, sampled_labels])
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.save_model()
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images_acgan/%d.png" % epoch)
        plt.close()

    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model_acgan/%s.json" % model_name
            weights_path = "saved_model_acgan/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "generator")
        save(self.discriminator, "discriminator")


if __name__ == '__main__':
    acgan = ACGAN()
    acgan.train(epochs=14000, batch_size=32, sample_interval=200)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 8, 8, 32)         

W0825 19:19:42.099804  9664 training.py:2197] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
W0825 19:19:44.551379  9664 training.py:2197] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?


0 [D loss: 3.321215, acc.: 40.62%, op_acc: 6.25%] [G loss: 3.467979]
1 [D loss: 3.357610, acc.: 45.31%, op_acc: 9.38%] [G loss: 3.140277]
2 [D loss: 3.337411, acc.: 54.69%, op_acc: 17.19%] [G loss: 3.284630]
3 [D loss: 3.088521, acc.: 75.00%, op_acc: 10.94%] [G loss: 3.235552]
4 [D loss: 3.159494, acc.: 73.44%, op_acc: 7.81%] [G loss: 2.779968]
5 [D loss: 2.831596, acc.: 87.50%, op_acc: 12.50%] [G loss: 3.256496]
6 [D loss: 2.973583, acc.: 87.50%, op_acc: 12.50%] [G loss: 2.976258]
7 [D loss: 2.881585, acc.: 89.06%, op_acc: 10.94%] [G loss: 2.876622]
8 [D loss: 2.905492, acc.: 92.19%, op_acc: 7.81%] [G loss: 2.869740]
9 [D loss: 2.838874, acc.: 89.06%, op_acc: 10.94%] [G loss: 2.699022]
10 [D loss: 2.997452, acc.: 95.31%, op_acc: 12.50%] [G loss: 2.717649]
11 [D loss: 2.543098, acc.: 96.88%, op_acc: 18.75%] [G loss: 2.603877]
12 [D loss: 2.687651, acc.: 96.88%, op_acc: 15.62%] [G loss: 2.469899]
13 [D loss: 2.673166, acc.: 100.00%, op_acc: 9.38%] [G loss: 2.832975]
14 [D loss: 2.668802

116 [D loss: 2.143001, acc.: 93.75%, op_acc: 37.50%] [G loss: 4.308603]
117 [D loss: 1.865226, acc.: 95.31%, op_acc: 50.00%] [G loss: 4.242300]
118 [D loss: 2.213240, acc.: 92.19%, op_acc: 37.50%] [G loss: 4.755268]
119 [D loss: 1.919468, acc.: 96.88%, op_acc: 40.62%] [G loss: 3.826535]
120 [D loss: 1.998042, acc.: 85.94%, op_acc: 42.19%] [G loss: 4.143795]
121 [D loss: 2.329183, acc.: 78.12%, op_acc: 35.94%] [G loss: 4.012620]
122 [D loss: 2.138139, acc.: 95.31%, op_acc: 37.50%] [G loss: 4.549945]
123 [D loss: 1.965271, acc.: 95.31%, op_acc: 50.00%] [G loss: 4.233677]
124 [D loss: 2.157821, acc.: 95.31%, op_acc: 34.38%] [G loss: 4.138605]
125 [D loss: 2.406135, acc.: 87.50%, op_acc: 29.69%] [G loss: 4.305609]
126 [D loss: 2.129192, acc.: 92.19%, op_acc: 39.06%] [G loss: 4.323100]
127 [D loss: 2.056161, acc.: 95.31%, op_acc: 35.94%] [G loss: 4.421327]
128 [D loss: 1.907866, acc.: 84.38%, op_acc: 45.31%] [G loss: 3.842242]
129 [D loss: 2.251694, acc.: 95.31%, op_acc: 28.12%] [G loss: 4.

230 [D loss: 1.644738, acc.: 93.75%, op_acc: 40.62%] [G loss: 4.810709]
231 [D loss: 1.498238, acc.: 96.88%, op_acc: 45.31%] [G loss: 4.760825]
232 [D loss: 1.598065, acc.: 90.62%, op_acc: 48.44%] [G loss: 4.542442]
233 [D loss: 1.919903, acc.: 76.56%, op_acc: 56.25%] [G loss: 4.288978]
234 [D loss: 1.581097, acc.: 87.50%, op_acc: 53.12%] [G loss: 4.465991]
235 [D loss: 1.747040, acc.: 87.50%, op_acc: 46.88%] [G loss: 4.357646]
236 [D loss: 1.476587, acc.: 98.44%, op_acc: 50.00%] [G loss: 4.215907]
237 [D loss: 1.676853, acc.: 93.75%, op_acc: 45.31%] [G loss: 4.480389]
238 [D loss: 1.725050, acc.: 87.50%, op_acc: 45.31%] [G loss: 4.413555]
239 [D loss: 1.978444, acc.: 90.62%, op_acc: 35.94%] [G loss: 4.932379]
240 [D loss: 1.788777, acc.: 89.06%, op_acc: 40.62%] [G loss: 4.656005]
241 [D loss: 1.884647, acc.: 84.38%, op_acc: 46.88%] [G loss: 4.669323]
242 [D loss: 2.010709, acc.: 84.38%, op_acc: 39.06%] [G loss: 4.178469]
243 [D loss: 1.502319, acc.: 96.88%, op_acc: 43.75%] [G loss: 4.

...

13459 [D loss: 0.708084, acc.: 54.69%, op_acc: 98.44%] [G loss: 0.751514]
13460 [D loss: 0.707271, acc.: 59.38%, op_acc: 98.44%] [G loss: 0.761033]
13461 [D loss: 0.739643, acc.: 48.44%, op_acc: 98.44%] [G loss: 0.725839]
13462 [D loss: 0.710770, acc.: 56.25%, op_acc: 98.44%] [G loss: 0.719488]
13463 [D loss: 0.733811, acc.: 59.38%, op_acc: 100.00%] [G loss: 0.748830]
13464 [D loss: 0.715847, acc.: 56.25%, op_acc: 98.44%] [G loss: 0.753949]
13465 [D loss: 0.728285, acc.: 53.12%, op_acc: 95.31%] [G loss: 0.733502]
13466 [D loss: 0.699445, acc.: 60.94%, op_acc: 98.44%] [G loss: 0.756917]
13467 [D loss: 0.694273, acc.: 57.81%, op_acc: 100.00%] [G loss: 0.708122]
13468 [D loss: 0.696757, acc.: 54.69%, op_acc: 100.00%] [G loss: 0.705259]
13469 [D loss: 0.885291, acc.: 46.88%, op_acc: 95.31%] [G loss: 0.744452]
13470 [D loss: 0.757964, acc.: 59.38%, op_acc: 96.88%] [G loss: 0.737063]
13471 [D loss: 0.729699, acc.: 40.62%, op_acc: 100.00%] [G loss: 0.705179]
13472 [D loss: 0.786117, acc.: 53.

13570 [D loss: 0.762568, acc.: 42.19%, op_acc: 100.00%] [G loss: 0.711059]
13571 [D loss: 0.780551, acc.: 45.31%, op_acc: 96.88%] [G loss: 0.719616]
13572 [D loss: 0.738500, acc.: 40.62%, op_acc: 100.00%] [G loss: 0.675728]
13573 [D loss: 0.774477, acc.: 50.00%, op_acc: 98.44%] [G loss: 0.760379]
13574 [D loss: 0.764290, acc.: 50.00%, op_acc: 98.44%] [G loss: 0.704272]
13575 [D loss: 0.784458, acc.: 60.94%, op_acc: 98.44%] [G loss: 0.741059]
13576 [D loss: 0.759973, acc.: 51.56%, op_acc: 98.44%] [G loss: 0.737395]
13577 [D loss: 0.808001, acc.: 42.19%, op_acc: 98.44%] [G loss: 0.774196]
13578 [D loss: 0.781815, acc.: 48.44%, op_acc: 96.88%] [G loss: 0.769944]
13579 [D loss: 0.704969, acc.: 53.12%, op_acc: 100.00%] [G loss: 0.738749]
13580 [D loss: 0.684196, acc.: 57.81%, op_acc: 100.00%] [G loss: 0.775463]
13581 [D loss: 0.702894, acc.: 53.12%, op_acc: 100.00%] [G loss: 0.748560]
13582 [D loss: 0.749892, acc.: 56.25%, op_acc: 98.44%] [G loss: 0.738395]
13583 [D loss: 0.835269, acc.: 50

13681 [D loss: 0.743607, acc.: 56.25%, op_acc: 98.44%] [G loss: 0.754338]
13682 [D loss: 0.709486, acc.: 51.56%, op_acc: 98.44%] [G loss: 0.735800]
13683 [D loss: 0.716229, acc.: 56.25%, op_acc: 98.44%] [G loss: 0.724049]
13684 [D loss: 0.754270, acc.: 48.44%, op_acc: 98.44%] [G loss: 0.759059]
13685 [D loss: 0.767500, acc.: 48.44%, op_acc: 98.44%] [G loss: 0.751997]
13686 [D loss: 0.707489, acc.: 45.31%, op_acc: 100.00%] [G loss: 0.714903]
13687 [D loss: 0.686241, acc.: 59.38%, op_acc: 100.00%] [G loss: 0.763167]
13688 [D loss: 0.794119, acc.: 42.19%, op_acc: 98.44%] [G loss: 0.724132]
13689 [D loss: 0.698548, acc.: 43.75%, op_acc: 100.00%] [G loss: 0.737269]
13690 [D loss: 0.741699, acc.: 45.31%, op_acc: 98.44%] [G loss: 0.717961]
13691 [D loss: 0.687725, acc.: 60.94%, op_acc: 100.00%] [G loss: 0.777551]
13692 [D loss: 0.860832, acc.: 31.25%, op_acc: 96.88%] [G loss: 0.722951]
13693 [D loss: 0.829811, acc.: 39.06%, op_acc: 96.88%] [G loss: 0.710488]
13694 [D loss: 0.689052, acc.: 54.

13791 [D loss: 0.702179, acc.: 53.12%, op_acc: 100.00%] [G loss: 0.765360]
13792 [D loss: 0.698249, acc.: 56.25%, op_acc: 98.44%] [G loss: 0.758132]
13793 [D loss: 0.772935, acc.: 67.19%, op_acc: 96.88%] [G loss: 0.731089]
13794 [D loss: 0.795050, acc.: 45.31%, op_acc: 96.88%] [G loss: 0.704704]
13795 [D loss: 0.696751, acc.: 57.81%, op_acc: 98.44%] [G loss: 0.772831]
13796 [D loss: 0.704439, acc.: 51.56%, op_acc: 100.00%] [G loss: 0.734213]
13797 [D loss: 0.716055, acc.: 46.88%, op_acc: 100.00%] [G loss: 0.750715]
13798 [D loss: 0.703653, acc.: 45.31%, op_acc: 100.00%] [G loss: 0.738897]
13799 [D loss: 0.712379, acc.: 48.44%, op_acc: 100.00%] [G loss: 0.772167]
13800 [D loss: 0.801804, acc.: 51.56%, op_acc: 96.88%] [G loss: 0.713093]
13801 [D loss: 0.704776, acc.: 60.94%, op_acc: 100.00%] [G loss: 0.713020]
13802 [D loss: 0.711378, acc.: 51.56%, op_acc: 100.00%] [G loss: 0.750254]
13803 [D loss: 0.700729, acc.: 50.00%, op_acc: 100.00%] [G loss: 0.723271]
13804 [D loss: 0.787805, acc.:

13902 [D loss: 0.724190, acc.: 53.12%, op_acc: 98.44%] [G loss: 0.737572]
13903 [D loss: 0.776840, acc.: 46.88%, op_acc: 98.44%] [G loss: 0.735993]
13904 [D loss: 0.732253, acc.: 50.00%, op_acc: 98.44%] [G loss: 0.730347]
13905 [D loss: 0.683723, acc.: 54.69%, op_acc: 100.00%] [G loss: 0.776834]
13906 [D loss: 0.772153, acc.: 59.38%, op_acc: 96.88%] [G loss: 0.763128]
13907 [D loss: 0.667956, acc.: 59.38%, op_acc: 98.44%] [G loss: 0.785034]
13908 [D loss: 0.754056, acc.: 53.12%, op_acc: 98.44%] [G loss: 0.779373]
13909 [D loss: 0.850219, acc.: 48.44%, op_acc: 93.75%] [G loss: 0.790248]
13910 [D loss: 0.769573, acc.: 65.62%, op_acc: 96.88%] [G loss: 0.768730]
13911 [D loss: 0.758566, acc.: 54.69%, op_acc: 98.44%] [G loss: 0.780133]
13912 [D loss: 0.741751, acc.: 45.31%, op_acc: 100.00%] [G loss: 0.755655]
13913 [D loss: 0.718790, acc.: 54.69%, op_acc: 98.44%] [G loss: 0.794694]
13914 [D loss: 0.773381, acc.: 51.56%, op_acc: 98.44%] [G loss: 0.743799]
13915 [D loss: 0.803261, acc.: 51.56